In [48]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

In [50]:
class StatsTracker:

    def __init__(self, array_shape):
        """
        array_shape: shape of the array whose stats we are tracking.
        compute: list of quantities we want to compute (by default all, but can omit some).
        """
        self.array_shape = array_shape
        self.n_samples = 0 # number of samples that have been pushed so far
        self.M1 = np.zeros(self.array_shape)
        self.M2 = np.zeros(self.array_shape)
        self.M3 = np.zeros(self.array_shape)
        self.M4 = np.zeros(self.array_shape)



    def push(self, sample):
        """Updates the running statistics to include the incoming sample.
        """

        # Increment number of samples
        prev_n_samples = self.n_samples
        self.n_samples += 1

        # Update everything else
        delta = sample - self.M1
        delta_n = delta/self.n_samples
        delta_n2 = delta_n*delta_n
        term1 = delta*delta_n*prev_n_samples
        self.M1 += delta_n
        self.M4 += term1*delta_n2*( (self.n_samples**2) - 3*self.n_samples + 3 ) + 6*delta_n2*self.M2 - 4*delta_n*self.M3
        self.M3 += term1*delta_n*(self.n_samples - 2) - 3*delta_n*self.M2
        self.M2 += term1



    def sample_size(self):
        """Returns the sample size.
        """
        return self.n_samples



    def mean(self):
        """Returns the current sample mean.
        """
        return self.M1



    def stdev(self):
        """Returns the current sample standard deviation.
        """
        return np.sqrt(self.variance())



    def variance(self):
        """Returns the current sample variance.
        """
        return self.M2/(self.n_samples - 1.0)



    def skewness(self):
        """Returns the current sample skewness.
        """
        return np.sqrt(self.n_samples)*self.M3/(self.M2**1.5)



    def kurtosis(self):
        """Returns the current sample kurtosis.
        """
        return self.n_samples*self.M4 / (self.M2*self.M2) - 3.0



    def clear(self):
        """Resets the tracker.
        """
        self.n_samples = 0
        self.M1 = np.zeros(self.array_shape)
        self.M2 = np.zeros(self.array_shape)
        self.M3 = np.zeros(self.array_shape)
        self.M4 = np.zeros(self.array_shape)
        


# Check

In [51]:
np.random.seed(0)
sample_size=3000
samples = np.random.normal(size=(3, 3, sample_size))

In [52]:
tracker = StatsTracker((3,3))
for j in range(sample_size):
    tracker.push(samples[:,:,j])

In [53]:
print(tracker.sample_size())

3000


In [54]:
print(tracker.mean())
print(np.mean(samples, axis=2))

[[-0.02762278 -0.00327172 -0.02238911]
 [-0.00886913  0.00973103  0.02161266]
 [ 0.01627541  0.00025343 -0.00655024]]
[[-0.02762278 -0.00327172 -0.02238911]
 [-0.00886913  0.00973103  0.02161266]
 [ 0.01627541  0.00025343 -0.00655024]]


In [60]:
print(tracker.variance())
print(np.var(samples, axis=2, ddof=1))

[[0.94180596 0.99917571 0.96985158]
 [1.01485015 0.96696622 0.98901217]
 [0.98803425 0.99709595 0.98431407]]
[[0.94180596 0.99917571 0.96985158]
 [1.01485015 0.96696622 0.98901217]
 [0.98803425 0.99709595 0.98431407]]


In [61]:
print(tracker.stdev())
print(np.std(samples, axis=2, ddof=1))

[[0.97046688 0.99958777 0.98481043]
 [1.00739771 0.9833444  0.99449091]
 [0.99399912 0.99854692 0.99212604]]
[[0.97046688 0.99958777 0.98481043]
 [1.00739771 0.9833444  0.99449091]
 [0.99399912 0.99854692 0.99212604]]


In [57]:
print(tracker.skewness())
print(skew(samples, axis=2))

[[ 0.00517277  0.04315719  0.04842729]
 [ 0.02046455  0.05361658  0.00577397]
 [-0.04682475 -0.02233904 -0.01176767]]
[[ 0.00517277  0.04315719  0.04842729]
 [ 0.02046455  0.05361658  0.00577397]
 [-0.04682475 -0.02233904 -0.01176767]]


In [58]:
print(tracker.kurtosis())
print(kurtosis(samples, axis=2))

[[-0.12220856 -0.00817616  0.08130489]
 [-0.04646253  0.06408941 -0.07376399]
 [ 0.14993777  0.14420643 -0.01303626]]
[[-0.12220856 -0.00817616  0.08130489]
 [-0.04646253  0.06408941 -0.07376399]
 [ 0.14993777  0.14420643 -0.01303626]]


# C++ source

In [ ]:
# #ifndef RUNNINGSTATS_H
# #define RUNNINGSTATS_H

# class RunningStats
# {
# public:
#     RunningStats();
#     void Clear();
#     void Push(double x);
#     long long NumDataValues() const;
#     double Mean() const;
#     double Variance() const;
#     double StandardDeviation() const;
#     double Skewness() const;
#     double Kurtosis() const;

#     friend RunningStats operator+(const RunningStats a, const RunningStats b);
#     RunningStats& operator+=(const RunningStats &rhs);

# private:
#     long long n;
#     double M1, M2, M3, M4;
# };

# #endif

# And here is the implementation file RunningStats.cpp.

# #include "RunningStats.h"
# #include <cmath>
# #include <vector>

# RunningStats::RunningStats() 
# {
#     Clear();
# }

# void RunningStats::Clear()
# {
#     n = 0;
#     M1 = M2 = M3 = M4 = 0.0;
# }

# void RunningStats::Push(double x)
# {
#     double delta, delta_n, delta_n2, term1;

#     long long n1 = n;
#     n++;
#     delta = x - M1;
#     delta_n = delta / n;
#     delta_n2 = delta_n * delta_n;
#     term1 = delta * delta_n * n1;
#     M1 += delta_n;
#     M4 += term1 * delta_n2 * (n*n - 3*n + 3) + 6 * delta_n2 * M2 - 4 * delta_n * M3;
#     M3 += term1 * delta_n * (n - 2) - 3 * delta_n * M2;
#     M2 += term1;
# }

# long long RunningStats::NumDataValues() const
# {
#     return n;
# }

# double RunningStats::Mean() const
# {
#     return M1;
# }

# double RunningStats::Variance() const
# {
#     return M2/(n-1.0);
# }

# double RunningStats::StandardDeviation() const
# {
#     return sqrt( Variance() );
# }

# double RunningStats::Skewness() const
# {
#     return sqrt(double(n)) * M3/ pow(M2, 1.5);
# }

# double RunningStats::Kurtosis() const
# {
#     return double(n)*M4 / (M2*M2) - 3.0;
# }

# RunningStats operator+(const RunningStats a, const RunningStats b)
# {
#     RunningStats combined;
    
#     combined.n = a.n + b.n;
    
#     double delta = b.M1 - a.M1;
#     double delta2 = delta*delta;
#     double delta3 = delta*delta2;
#     double delta4 = delta2*delta2;
    
#     combined.M1 = (a.n*a.M1 + b.n*b.M1) / combined.n;
    
#     combined.M2 = a.M2 + b.M2 + 
#                   delta2 * a.n * b.n / combined.n;
    
#     combined.M3 = a.M3 + b.M3 + 
#                   delta3 * a.n * b.n * (a.n - b.n)/(combined.n*combined.n);
#     combined.M3 += 3.0*delta * (a.n*b.M2 - b.n*a.M2) / combined.n;
    
#     combined.M4 = a.M4 + b.M4 + delta4*a.n*b.n * (a.n*a.n - a.n*b.n + b.n*b.n) / 
#                   (combined.n*combined.n*combined.n);
#     combined.M4 += 6.0*delta2 * (a.n*a.n*b.M2 + b.n*b.n*a.M2)/(combined.n*combined.n) + 
#                   4.0*delta*(a.n*b.M3 - b.n*a.M3) / combined.n;
    
#     return combined;
# }

# RunningStats& RunningStats::operator+=(const RunningStats& rhs)
# { 
#         RunningStats combined = *this + rhs;
#         *this = combined;
#         return *this;
# }